<a href="https://colab.research.google.com/github/klajosw/python/blob/master/KL_sqlite_minta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SqLite használata

In [0]:
## Modulok impontálása
import sqlite3
import os
import hashlib

In [0]:
## Belső függveny eljárások definiálása
def collate_reverse(string1, string2):
    if string1 == string2:
        return 0
    elif string1 < string2:
        return 1
    else:
        return -1

## hashlib.md5 / hashlib.sha224 /// Titkosítás GDPR!!!
def md5sum(t):
    if type(t) == int:
        t = str(t)
    t = t.encode()    
    return hashlib.md5(t).hexdigest()  
    
def sha224sum(t):
    if type(t) == int:
        t = str(t)
    t = t.encode()    
    return hashlib.sha224(t).hexdigest()   

## Memoria adatbázis, tábla használati módszer

In [0]:
## Memoria adatbázis / tábla használata
sqlite3.enable_shared_cache(True) ### 

con = sqlite3.connect(":memory:")
con.isolation_level = None

con.create_collation("reverse", collate_reverse)  ## fuggvény használat

con.create_function("md5", 1, md5sum)

cur = con.cursor()

cur.execute("select md5(?)", ("foo",))
print(cur.fetchone()[0])

cur.execute("create table test(x)")
cur.executemany("insert into test(x) values (?)", [("a",), ("b",), ("d",) , ("c",)])
## cur.execute("select x from test order by x collate reverse")

cur.execute("select x from test order by x asc")

print('---- Kiíratás 0.  ----\n')
for row in cur:
    print(row)
con.close()

acbd18db4cc2f85cedef654fccc4a4d8
---- Kiíratás 0.  ----

('a',)
('b',)
('c',)
('d',)


## Állomány adatbázis használata

In [37]:
## Állomány adatbázis
DB_FILE = "/content/sample_data/mydb.db"  # adatbázis neve
newdb=0

if os.path.exists(DB_FILE ):  ## adatbázis állomány létezik akkor törlés
   os.remove(DB_FILE)
   newdb = 1
   print('Törölve')


con = sqlite3.connect(DB_FILE)  ## ha nem létezik létrehozza

con.create_function("md5", 1, md5sum)

cur = con.cursor()

if newdb == 1 :  ## Ha új adatbázis akkor a táblát léátrekell hozni
    cur.execute(""" create table people (
      name_last  varchar(20), 
      age        integer)  """)
    
    


## Rekordok beszúrása / INSERT
cur.execute("insert into people (name_last, age) values ('Lajos',   57)")    
cur.execute("insert into people (name_last, age) values ('Béla',     53)")   
cur.execute("insert into people (name_last, age) values ('Ildikó',     54)")  

con.commit()  ## Beírás véglegesítése

newPeople = (
    ('Dani' , 22),
    ('Tomi' , 21)
  )

for person in newPeople:
    cur.execute("insert into people (name_last, age) values (?, ?)", person)

# The changes will not be saved unless the transaction is committed explicitly:
con.commit()  ## Beírás véglegesítése

##-----------------

print('---- Kiíratás 11  ----\n')
cur.execute("select * from people ")
for row in cur:
    for mezo in row:
        print(mezo)
        
print('---- Kiíratás 2.  ----\n')        
cur.execute("select * from people ")
for row in cur:        
        print('%s is %d years old.' % (row[0], row[1]))
##
print("Most törlök ", cur.execute("delete from people where name_last = 'Tomi' ").rowcount, "rekordot/sort.") 

print('---- Kiíratás 3.  ----\n')
cur.execute(" select name_last, age,  md5(name_last) as name_md5 from people ")

print("with column names:", cur.fetchone()[0])
print("with column names:", cur.fetchone())

print(cur.fetchall())  ## Összes további kiíratása


print('---- Kiíratás 4.  ----\n')
FIELD_MAX_WIDTH = 20  ## mezők maximális hossz beállítása
cur.execute(" select name_last, age,  md5(name_last) as name_md5 from people ")

# Fejléc
for fieldDesc in cur.description:
    print(fieldDesc[0].ljust(FIELD_MAX_WIDTH), end=' ')
print()          # Fejléc vége  
print('-' * 78)  # elválasztó sor

# For each row, print the value of each field left-justified within
# the maximum possible width of that field.
fieldIndices = range(len(cur.description))
for row in cur:
    for fieldIndex in fieldIndices:
        fieldValue = str(row[fieldIndex])
        print(fieldValue.ljust(FIELD_MAX_WIDTH), end=' ')

    print() # kiíratás végén sorvégjel kitétele


cur.close()
con.close()

OperationalError: ignored

In [34]:
chmod 664 /content/sample_data/

SyntaxError: ignored